First, we load the data.

In [14]:
import pandas as pd

whole_train_df = pd.read_csv('uji_wifi/UJIndoorLoc/trainingData.csv')
test_df = pd.read_csv('uji_wifi/UJIndoorLoc/ValidationData.csv')

We split the training data into training and validation sets using the USERID field.

In [15]:
train_mask = whole_train_df['USERID'] <= 13
train_df = whole_train_df[train_mask]
val_df = whole_train_df[~train_mask]
print(train_df.shape, val_df.shape)

(15647, 529) (4290, 529)


We put the data into NumPy arrays and set to -100dBm the values corresponding to absent AP.

In [16]:
import numpy as np

train_X = train_df.iloc[:,0:520].to_numpy()
train_building = train_df["BUILDINGID"].to_numpy(dtype=np.int64)
train_floor = train_df["FLOOR"].to_numpy(dtype=np.int64)
train_long = train_df["LONGITUDE"].to_numpy()
train_lat = train_df["LATITUDE"].to_numpy()

val_X = val_df.iloc[:,0:520].to_numpy()
val_building = val_df["BUILDINGID"].to_numpy(dtype=np.int64)
val_floor = val_df["FLOOR"].to_numpy(dtype=np.int64)
val_long = val_df["LONGITUDE"].to_numpy()
val_lat = val_df["LATITUDE"].to_numpy()

test_X = test_df.iloc[:,0:520].to_numpy()
test_building = test_df["BUILDINGID"].to_numpy(dtype=np.int64)
test_floor = test_df["FLOOR"].to_numpy(dtype=np.int64)
test_long = test_df["LONGITUDE"].to_numpy()
test_lat = test_df["LATITUDE"].to_numpy()


train_X[train_X == 100] = -110
val_X[val_X == 100] = -110
test_X[test_X == 100] = -110

Some evaluation metrics

In [17]:
from sklearn.metrics import mean_squared_error


def accuracy(pred_Y, true_Y):
    return np.sum(pred_Y == true_Y)/len(true_Y)

def distance_rmse(pred_long, pred_lat, true_long, true_lat):
    sq_dist = (pred_long - true_long)**2 + (pred_lat - true_lat)**2
    return np.sqrt(np.sum(sq_dist)/len(sq_dist))

def dist_mean_error(pred_long, pred_lat, true_long, true_lat):
    dist = np.sqrt((pred_long - true_long)**2 + (pred_lat - true_lat)**2)
    return np.sum(dist)/len(pred_long)

The following architecture is used for longitude/latitude estimation

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, num_outputs=2):
        super(MLP, self).__init__()
        
        self.linear1 = nn.Linear(520, 256) 
        self.bn1 = nn.BatchNorm1d(256)

        self.linear2 = nn.Linear(256, 256)
        self.bn2 = nn.BatchNorm1d(256)
        
        self.linear3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        
        self.linear4 = nn.Linear(128, 128)
        self.bn4 = nn.BatchNorm1d(128)
        
        self.linear5 = nn.Linear(128, 64)
        self.bn5 = nn.BatchNorm1d(64)
        
        self.linear6 = nn.Linear(64, 32)
        self.bn6 = nn.BatchNorm1d(32)
    

        self.final = nn.Linear(32, num_outputs)
      
    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.linear1(x)))
        x = F.leaky_relu(self.bn2(self.linear2(x)))
        x = F.leaky_relu(self.bn3(self.linear3(x)))
        x = F.leaky_relu(self.bn4(self.linear4(x)))
        x = F.leaky_relu(self.bn5(self.linear5(x)))
        x = F.leaky_relu(self.bn6(self.linear6(x)))
    
        x = self.final(x)
        return x

In [19]:
import time
import copy

def train(model, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs, patience=10, transform=None):
    data_loaders = {'train': train_loader, 'val': val_loader}
    data_lengths = {'train': len(train_loader), 'val': len(val_loader)}
    best_val_loss = None
    epochs_no_improve = 0
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}')
        epoch_start = time.time()
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            running_loss = 0.0
            for i, data in enumerate(data_loaders[phase]):   
                inputs, targets = data
                inputs, targets = inputs.cuda(),targets.cuda()
                if phase == 'train' and transform is not None:
                    inputs = transform(inputs)

                optimizer.zero_grad()

                outputs = model(inputs).squeeze(-1) 
                loss = criterion(outputs,targets)  
                
                if phase == 'train':
                    loss.backward() 
                    optimizer.step() 

                running_loss += loss.item()
            
            epoch_loss = running_loss/data_lengths[phase]
            if phase == 'train':
                print(f'Training loss : {epoch_loss}')
                print(f'Learning rate : {optimizer.param_groups[0]["lr"]}')
            else:
                scheduler.step(epoch_loss)
                print(f'Validation loss : {epoch_loss}')
                if best_val_loss is None or epoch_loss < best_val_loss:
                    epochs_no_improve = 0
                    best_val_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                else:
                    epochs_no_improve += 1
                    if epochs_no_improve >= patience:
                        print('Early stopping')
                        model.load_state_dict(best_model_wts)
                        return model
                
        print(f'Epoch duration : {time.time() - epoch_start}')
    model.load_state_dict(best_model_wts)
    return model

In [20]:
def predict(model, X):
    model.eval()
    with torch.no_grad():
        inputs = torch.tensor(normalize_rssi(X), dtype=torch.float32).cuda()
        outputs = model(inputs).squeeze(-1)
        return outputs.cpu().detach().numpy()

Inputs and outputs are normalized. If outputs were not normalized, the performance are not good because of the batch normalization.

In [21]:
def normalize_rssi(rssi):
    return (110 + rssi)/110

def normalize_longitude(longitude):
    return (-longitude - 7299) / 395

def denormalize_longitude(norm_long):
    return -(norm_long * 395 + 7299)

def normalize_latitude(latitude):
    return  (latitude -  4864745)/218

def denormalize_latitude(norm_lat):
    return 218*norm_lat + 4864745

White Gaussian noise is added to inputs in order to reduce overfitting.

In [22]:
class AddGaussianNoise:
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()).cuda() * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

PyTorch data loaders

In [23]:
from torch.utils.data import Dataset, DataLoader, TensorDataset


long_tensor = torch.tensor(normalize_longitude(train_long), dtype=torch.float32)
lat_tensor = torch.tensor(normalize_latitude(train_lat), dtype=torch.float32)

target_tensor = torch.cat((long_tensor.view(-1,1), lat_tensor.view(-1,1)), 1)

train_loader = DataLoader(TensorDataset(torch.tensor(normalize_rssi(train_X), dtype=torch.float32), target_tensor), batch_size=128, shuffle=True, num_workers=4, pin_memory=True)


long_tensor = torch.tensor(normalize_longitude(val_long), dtype=torch.float32)
lat_tensor = torch.tensor(normalize_latitude(val_lat), dtype=torch.float32)

target_tensor = torch.cat((long_tensor.view(-1,1), lat_tensor.view(-1,1)), 1)

val_loader = DataLoader(TensorDataset(torch.tensor(normalize_rssi(val_X), dtype=torch.float32), target_tensor), batch_size=128, shuffle=True, num_workers=4, pin_memory=True)

Training of the model

In [24]:
from torch.optim import lr_scheduler
import torch.optim as optim

mlp = MLP(num_outputs=2).cuda()

optimizer = optim.Adam(mlp.parameters(), lr=0.001)
criterion = nn.MSELoss()
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, verbose=True)

mlp = train(mlp, train_loader, val_loader, optimizer, criterion, scheduler, num_epochs=200, patience=20, transform=None)

Epoch 1
Training loss : 0.08317833221555362
Learning rate : 0.001
Validation loss : 0.0038095085867954526
Epoch duration : 1.5143654346466064
Epoch 2
Training loss : 0.0030909818114440253
Learning rate : 0.001
Validation loss : 0.0024131135672659557
Epoch duration : 1.520348310470581
Epoch 3
Training loss : 0.002246985155653693
Learning rate : 0.001
Validation loss : 0.0028846086660290465
Epoch duration : 1.5291497707366943
Epoch 4
Training loss : 0.0017390344907128351
Learning rate : 0.001
Validation loss : 0.002067970029790612
Epoch duration : 1.6027476787567139
Epoch 5
Training loss : 0.0015796988713176452
Learning rate : 0.001
Validation loss : 0.001591394521871253
Epoch duration : 1.7394757270812988
Epoch 6
Training loss : 0.0013378017025477275
Learning rate : 0.001
Validation loss : 0.0015606194022385513
Epoch duration : 1.5787162780761719
Epoch 7
Training loss : 0.0012396121249965777
Learning rate : 0.001
Validation loss : 0.001972234948291717
Epoch duration : 1.5911250114440918

Exception in thread Thread-127:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/opt/conda/lib/python3.7/site-packages/torch/multiprocessing/reductions.py", line 282, in rebuild_storage_fd
    fd = df.detach()
  File "/opt/conda/lib/python3.7/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/opt/conda/lib/python3.7/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().

KeyboardInterrupt: 

Evaluation on the validation set

In [ ]:
val_pred = predict(mlp, val_X)
val_pred_long = denormalize_longitude(val_pred[:,0])
val_pred_lat = denormalize_latitude(val_pred[:,1])

long_rmse = mean_squared_error(val_pred_long, val_long, squared=False)
lat_rmse = mean_squared_error(val_pred_lat, val_lat, squared=False)
dist_rmse = distance_rmse(val_pred_long, val_pred_lat, val_long, val_lat)
dist_me = dist_mean_error(val_pred_long, val_pred_lat, val_long, val_lat)

print('Val longitude RMSE : ', long_rmse)
print('Val latitude RMSE : ', lat_rmse)
print('Val distance RMSE : ', dist_rmse)
print('Val distance ME : ', dist_me)

Evaluation on the test set

In [ ]:
test_pred = predict(mlp, test_X)
test_pred_long = denormalize_longitude(test_pred[:,0])
test_pred_lat = denormalize_latitude(test_pred[:,1])

long_rmse = mean_squared_error(test_pred_long, test_long, squared=False)
lat_rmse = mean_squared_error(test_pred_lat, test_lat, squared=False)
dist_rmse = distance_rmse(test_pred_long, test_pred_lat, test_long, test_lat)
dist_me = dist_mean_error(test_pred_long, test_pred_lat, test_long, test_lat)

print('Test longitude RMSE : ', long_rmse)
print('Test latitude RMSE : ', lat_rmse)
print('Test distance RMSE : ', dist_rmse)
print('Test distance ME : ', dist_me)